In [13]:
import pandas as pd
import json as json
import matplotlib.pyplot as plt
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/Users/lucas/anaconda/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['f', 'test', 'split']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [14]:
df = pd.read_json('./data.json')

In [15]:
js = {}
with open('./data.json') as f:
    js  = json.loads(f.read())
df = pd.DataFrame(js['result']['records'])
df.head(0)

,VIP_sales,VIP_trn_count,_id,emerging_usd,giftncase_usd,month,selftreat_usd,sold_count_Bar_selftreat,sold_count_Biscuits_emerging,sold_count_BulkCase_giftncase_in_gram,...,sold_count_Giftbox_giftncase,sold_count_Icecream_emerging,sold_count_IndividualWrappedChocolate_selftreat_in _gram,sold_count_KidsRange_selftreat,sold_count_Pastry_emerging,sold_count_PretzelAndSmallChocolate_selftreat,sold_count_Tablet_selftreat,total_usd,trn_count,year


### change delete ',' and column dtype

In [16]:
for i in range(df.shape[0]):
    for column in df.columns:
        df.loc[i,column] =  str(df.loc[i,column]).replace(',','')

In [17]:
for column in df.columns:
    if column != 'month':
        df.loc[:,column] =  df.loc[:,column].astype('float64')

### Plot - sales amount via month

In [18]:
# df.plot(x='month',y='giftncase_usd')
# df.plot(x='month',y='emerging_usd')
# df.plot(x='month',y='selftreat_usd')

In [19]:
df.to_csv('data.csv')

# Generate labels

## Total number of unit sold should change from g to pocket

In [20]:
count = pd.read_csv('./label_count.csv')[['giftncase_count','emerging_count','selftreat_count']]
tr_y = df[['giftncase_usd','emerging_usd','selftreat_usd']]
tr_y = tr_y.join(count)
tr_y.head(0)

,giftncase_usd,emerging_usd,selftreat_usd,giftncase_count,emerging_count,selftreat_count


# Load features data from different sources on internet

- google index: https://trends.google.com/trends/explore?date=all&q=godiva  
- data website: https://insights.ceicdata.com/Untitled-insight/myseries
- festival website: https://www.gov.hk/sc/about/abouthk/holiday/2017.htm  

In [21]:
# tr=df[['total_usd','sold_count_Drinks_emerging']]
google_index=pd.DataFrame(pd.read_csv('./googleindex_a2.csv')['index'])
OccupancyRate = pd.DataFrame(pd.read_csv('./OccupancyRateHK.csv')['OccupancyRate'])
value = pd.DataFrame(pd.read_csv('./weibo_month.csv')['value'])
FestivalCount = pd.DataFrame(pd.read_csv('./FestivalCountHK.csv')['FestivalCount'])
temp = pd.DataFrame(pd.read_csv('./hongkong_temp.csv')[['average_highest_temp','average_temp']])
tr = google_index.join([value,FestivalCount,temp])
tr = google_index.join([temp])
feature_label = OccupancyRate.join([google_index,value,FestivalCount,temp,tr_y])
# feature_label.to_csv('feature_label.csv',index=False)
# tr.to_csv('features.csv',index=False)


## StandardScaler

In [22]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(tr.iloc[:19,:])
tr_bf = pd.DataFrame(sc.transform(tr.iloc[:19,:]),columns=tr.columns)
tr_af = pd.DataFrame(sc.transform(tr.iloc[19:,:]),columns=tr.columns)
tr = pd.concat([tr_bf,tr_af],axis = 0,ignore_index=True)
tr.head(0)

,index,average_highest_temp,average_temp


# Training  
- LinearRegression for sales amount usd  
- SVR for units amount count  

In [23]:
from sklearn.linear_model import Lasso,LinearRegression
from sklearn.svm import LinearSVR,SVR
clfs = []
x_columns_name = list(tr.columns)
y_columns_name = list(tr_y.columns)
def train(tr,tr_y):
    print 'train ========'
    print list(tr.columns)
    for column in tr_y.columns: 
        clf = Lasso()
        clf = LinearRegression()
#         clf = SVR(C=1e3, gamma=1e-1 , epsilon = 0.1)
#         clf = LinearSVR()
        clf.fit(tr,tr_y[column])
        clfs.append(clf)
        
#         print column,'coef_',list(np.round(clf.coef_,0))
#         coef = pd.Series(clf.coef_, index = tr.columns)# .coef_ 可以返回经过学习后的所有 feature 的参数。
#         print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")
#         imp_coef = pd.concat([coef.sort_values()])
#         matplotlib.rcParams['figure.figsize'] = (18.0, 9.0)
#         imp_coef.plot(kind = "barh")
#         plt.title("Coefficients in the Lasso Model")
    print '====end======'
    return clfs
def test(clfs,ts,ts_y):
    print 'test========'
    preds=[]
    for i in range(ts_y.shape[1]):
#     for clf in clfs:
        pred = clfs[i].predict(ts)
#         print '\t\tTRUE:',ts_y.iloc[:,i].values
        print '\t\tPred:',np.round(pred)
        sse = square(ts_y.iloc[:,i].values-pred).sum()/np.square(ts_y.iloc[:,i].values).sum()
        print y_columns_name[i],'SSE_rate:\n-',sse
        preds.append(pred)
    print '====end======='
    return preds

split = 19
xx = tr.iloc[:split,:]
yy = tr_y.iloc[:split,:]
xxx = tr.iloc[split:22,:]
yyy = tr_y.iloc[split:22,:]
clfs = train(xx,yy)

preds = test(clfs,xxx,yyy)

train ========
['index', 'average_highest_temp', 'average_temp']
====end======
test========
		Pred: [ 282.  283.  507.]
giftncase_usd SSE_rate:
- 0.443037472572
		Pred: [ 577.  540.  521.]
emerging_usd SSE_rate:
- 0.00829122139675
		Pred: [ 112.  110.  134.]
selftreat_usd SSE_rate:
- 0.0851415836678
		Pred: [ 84225.  79668.  78057.]
giftncase_count SSE_rate:
- 0.00417713815399
		Pred: [ 91889.  86139.  79441.]
emerging_count SSE_rate:
- 0.00332171907766
		Pred: [ 10198.  10139.  11140.]
selftreat_count SSE_rate:
- 0.0209286514382
====end=======


# compare labels trends of  2016/11-2017/3 and 2015/11-2016/3

In [24]:
# print 'test========'
# preds=[]
# cloumns = tr_y.columns
# for i in range(6):
# #     for clf in clfs:
#     pred = clfs[i].predict(tr.iloc[22:,:])
#     plt.plot(range(5), pred,'b')
#     plt.title(cloumns[i])
#     plt.plot(range(5), tr_y.iloc[10:15,i], 'r--')
#     plt.show()
#     plt.clf()
#     plt.cla()
#     plt.close()
# #     pd.DataFrame(pred).plot()
#     print '\t\tpre:',pred
#     print '\t\ttrue:',list(tr_y.iloc[10:15,i])
#     preds.append(pred)
# print '====end======='